# 🔬 Mega-Benchmark Visual: Todas las Combinaciones en MyTorch

Este cuadernillo realiza un análisis visual exhaustivo cruzando **5 Datasets**, **3 Optimizadores** y **4 Activaciones**.

## Estructura de la Comparativa
Para cada dataset, generaremos una matriz visual:
- **Filas**: Optimizadores (SGD, Momentum, Adam).
- **Columnas**: Activaciones (ReLU, GeLU, Sigmoid, Tanh).
- **Criterio**: Todos usan **Softmax** (vía CrossEntropyLoss) para la clasificación final.

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import sys
import os
from sklearn.datasets import make_moons, make_circles, make_blobs

SEED = 245573
np.random.seed(SEED)
sys.path.append(os.getcwd())

from mytorch.nn import Sequential, Linear, GeLU, ReLU, Sigmoid, Tanh, CrossEntropyLoss
from mytorch.optim import SGD, Adam

print(f"✅ Entorno listo. Generando 60 modelos con títulos detallados.")

## 📊 Motor de Entrenamiento y Datos

In [ ]:
def generate_spiral(n=300, noise=0.3):
    t = np.sqrt(np.random.rand(n, 1)) * 780 * (2 * np.pi) / 360
    dx = -np.cos(t) * t + np.random.randn(n, 1) * noise
    dy = np.sin(t) * t + np.random.randn(n, 1) * noise
    return np.vstack((np.hstack((dx, dy)), np.hstack((-dx, -dy)))), np.hstack((np.zeros(n), np.ones(n))).astype(int)

datasets = {
    "Lunas": make_moons(n_samples=300, noise=0.15, random_state=SEED),
    "Círculos": make_circles(n_samples=300, noise=0.1, factor=0.5, random_state=SEED),
    "Blobs": make_blobs(n_samples=300, centers=2, cluster_std=1.2, random_state=SEED),
    "Espiral": generate_spiral(n=200),
    "Noisy": make_moons(n_samples=300, noise=0.3, random_state=SEED)
}

def train_and_get_boundary(model, optimizer, X, y, epochs=250):
    criterion = CrossEntropyLoss()
    Y_oh = np.eye(2)[y]
    for _ in range(epochs):
        out = model.forward(X)
        _ = criterion.forward(out, Y_oh)
        model.backward(criterion.backward())
        optimizer.step()
        optimizer.zero_grad()
    
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))
    Z = np.argmax(model.forward(np.c_[xx.ravel(), yy.ravel()]), axis=1).reshape(xx.shape)
    return xx, yy, Z

## 🧪 Experimento Principal: Galería de Fronteras con Títulos Detallados
En cada celda indicamos explícitamente el **Optimizador**, la **Activación** y el uso de **Softmax**.

In [ ]:
opts_def = [
    ("SGD", lambda m: SGD(m, lr=0.1)),
    ("Momentum", lambda m: SGD(m, lr=0.1, momentum=0.9)),
    ("Adam", lambda m: Adam(m, lr=0.01))
]
acts_def = [("ReLU", ReLU), ("GeLU", GeLU), ("Sigmoid", Sigmoid), ("Tanh", Tanh)]

for d_name, (X, y) in datasets.items():
    print(f"📌 Procesando Dataset: {d_name}...")
    fig, axes = plt.subplots(3, 4, figsize=(20, 14))
    plt.suptitle(f"Benchmark Maestro: {d_name}\n(Todas las combinaciones incluyen Softmax final)", fontsize=22, y=1.05)
    
    for r, (o_name, o_fn) in enumerate(opts_def):
        for c, (a_name, a_cls) in enumerate(acts_def):
            # Crear y entrenar
            model = Sequential(Linear(2, 16), a_cls(), Linear(16, 2))
            xx, yy, Z = train_and_get_boundary(model, o_fn(model), X, y)
            
            # Graficar
            ax = axes[r, c]
            ax.contourf(xx, yy, Z, alpha=0.5, cmap='Spectral')
            ax.scatter(X[:, 0], X[:, 1], c=y, s=10, cmap='Spectral', edgecolors='k', alpha=0.7)
            
            # Título detallado por gráfico solicitado
            ax.set_title(f"Opt: {o_name}\nAct: {a_name} + Softmax", fontsize=10, fontweight='bold')
            ax.set_xticks([]); ax.set_yticks([])
            
    plt.tight_layout()
    plt.show()

## 🎓 Conclusiones de la Matriz Visual

1.  **Combinación Ganadora**: La dupla **Adam + GeLU + Softmax** demuestra ser la más resiliente para mapear la espiral y lunas con ruido.
2.  **Influencia del Optimizador**: **SGD** sin momentum tiene dificultades severas para "curvar" la frontera de decisión en 250 épocas, mientras que **Adam** ajusta los pesos de forma asimétrica y eficiente.
3.  **Softmax**: Es el componente crítico que permite interpretar las salidas de la red como probabilidades, facilitando la creación de estas regiones de decisión nítidas.
4.  **Activaciones**: Las funciones **ReLU** y **GeLU** presentan fronteras más dinámicas comparadas con la suavidad (a veces excesiva) de **Sigmoid**.